# Model soup

In [1]:
# 기본코드
import inspect
import os
import sys
import time
import numpy as np

import torch
from mmcv import Config
from mmseg.datasets import build_dataloader, build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import single_gpu_test
from mmcv.runner import load_checkpoint, load_state_dict
from mmcv.parallel import MMDataParallel

def uniform_soup(cfg, model, checkpoint_paths, device = "cpu", by_name = False):
    try:
        import torch
    except:
        print("If you want to use 'Model Soup for Torch', please install 'torch'")
        return model
    
    dataset = build_dataset(cfg.data.val)
    data_loader = build_dataloader(
            dataset,
            samples_per_gpu=1,
            workers_per_gpu=cfg.data.workers_per_gpu,
            dist=False,
            shuffle=False)  
    
    
    model = model.to(device)
    model_dict = model.state_dict()
    soups = {key:[] for key in model_dict}
    checkpoint = {}
    for i, checkpoint_path in enumerate(checkpoint_paths):
        checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')
        weight_dict = checkpoint['state_dict']
        for k, v in weight_dict.items():
            soups[k].append(v)
    if 0 < len(soups):
        soups = {k:(torch.sum(torch.stack(v), axis = 0) / len(v)).type(v[0].dtype) for k, v in soups.items() if len(v) != 0}
        model_dict.update(soups)
        model.load_state_dict(model_dict)
    
    load_state_dict(model, model_dict)
    model.CLASSES = dataset.CLASSES
    model = MMDataParallel(model.cuda(), device_ids=[0])
    output = single_gpu_test(model, data_loader)
    eval_kwargs = {}
    eval_kwargs.update(metric=['mIoU'])
    metric = dataset.evaluate(output, **eval_kwargs)
    print(f"mIoU: {metric['mIoU']}")
    
    return model, checkpoint

def greedy_soup(cfg, model_ori, checkpoint_paths, device = "cpu"):
    try:
        import torch
    except:
        print("If you want to use 'Model Soup for Torch', please install 'torch'")
        return model_ori
    
    dataset = build_dataset(cfg.data.val)
    data_loader = build_dataloader(
            dataset,
            samples_per_gpu=1,
            workers_per_gpu=cfg.data.workers_per_gpu,
            dist=False,
            shuffle=False)  
    
    
    result = []
    checkpoint = {}
    for i, checkpoint_path in enumerate(checkpoint_paths):
        model = model_ori.to(device)
        checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')
        model.CLASSES = dataset.CLASSES
        model = MMDataParallel(model.cuda(), device_ids=[0])
        output = single_gpu_test(model, data_loader)
        eval_kwargs = {}
        eval_kwargs.update(metric=['mIoU'])
        metric = dataset.evaluate(output, **eval_kwargs)
        result.append((metric['mIoU'],checkpoint_path))
        print(f"리스트에 {i}번째 mIoU {metric['mIoU']}저장")
    
    result.sort(key = lambda x : x[0], reverse = True)
    print(f"리스트 정렬")
    print(result)
    
    model = model_ori.to(device)
    model_dict = model.state_dict()
    pre_metric_value = 0
    pre_weight_dict = {}
    for i, (mIoU, checkpoint_path) in enumerate(result):
        model = model_ori.to(device)
        soups = {key:[] for key in model_dict}
        now_model_dict = model_dict
        if i == 0:
            checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')
            pre_metric_value = mIoU
            pre_weight_dict = checkpoint['state_dict']
            print("soup 모델에 가장 높은 mIou를 가진 checkpoint가 추가되었습니다")
            print(f"추가된 checkpoint_path: {checkpoint_path}")
            print(f"현재 최고 mIoU: {pre_metric_value}")
        else:
            checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')
            weight_dict = checkpoint['state_dict']
            
            for k, v in pre_weight_dict.items():
                soups[k].append(v)
            for k, v in weight_dict.items():
                soups[k].append(v)    
            if 0 < len(soups):
                soups = {k:(torch.sum(torch.stack(v), axis = 0) / len(v)).type(v[0].dtype) for k, v in soups.items() if len(v) != 0}
                now_model_dict.update(soups)
                
                
            load_state_dict(model, now_model_dict)
            model.CLASSES = dataset.CLASSES
            model = MMDataParallel(model.cuda(), device_ids=[0])
            output = single_gpu_test(model, data_loader)
            eval_kwargs = {}
            eval_kwargs.update(metric=['mIoU'])
            metric = dataset.evaluate(output, **eval_kwargs)
            
            if metric['mIoU'] >= pre_metric_value:
                pre_metric_value = metric['mIoU']
                pre_weight_dict = now_model_dict
                print("soup 모델에 새로운 checkpoint가 추가되었습니다")
                print(f"추가된 checkpoint_path: {checkpoint_path}")
                print(f"현재 최고 mIoU: {pre_metric_value}")
            else:
                print("이번 체크 포인트는 soup 모델에 추가되지 않았습니다")
                print(f"이번 checkpoint_path: {checkpoint_path}")
                print(f"현재 최고 mIoU: {pre_metric_value}, 이번 mIou {metric['mIoU']}")
            
    model = model_ori.to(device)
    load_state_dict(model, pre_weight_dict)
    return model, checkpoint

0. 모델 & checkpoint 가져오기

In [2]:
################ model cfg path 적기 ################
cfg= Config.fromfile('/opt/ml/input/level2_semanticsegmentation_cv-level2-cv-09/mmsegmentation/work_dirs/segformer_mit-b0_512x512_160k_ade20k/segformer_mit-b0_512x512_160k_ade20k.py')
################ model cfg path 적기 ################
model = build_segmentor(cfg.model)

################ soup할 checkpoint path 적기 ################
checkpoint_paths = [
    '/opt/ml/input/level2_semanticsegmentation_cv-level2-cv-09/mmsegmentation/work_dirs/segformer_mit-b0_512x512_160k_ade20k/iter_96000.pth',
    '/opt/ml/input/level2_semanticsegmentation_cv-level2-cv-09/mmsegmentation/work_dirs/segformer_mit-b0_512x512_160k_ade20k/iter_108000.pth',
    '/opt/ml/input/level2_semanticsegmentation_cv-level2-cv-09/mmsegmentation/work_dirs/segformer_mit-b0_512x512_160k_ade20k/iter_120000.pth',
]
################ soup할 checkpoint path 적기 ################
device = "cpu"

/opt/ml/input/level2_semanticsegmentation_cv-level2-cv-09/mmsegmentation/mmseg/models/backbones/mit.py:365: UserWarning: DeprecationWarning: pretrained is deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrained is deprecated, '


1. uniform soup

In [4]:
################ save dir path 적기 ################
save_dir_path = './soup/'
name = 'test' # soup 이름 적기
################ save dir path 적기 ################

import wandb
wandb.init()
print("\n[Uniform Soup]")
uniform_model, checkpoint = uniform_soup(cfg, model, checkpoint_paths, device = device)
uniform_dict = checkpoint
uniform_dict['state_dict'] = uniform_model.state_dict()

torch.save(uniform_dict, save_dir_path+f'uniform_model_soup_{name}.pth')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: syoon6682 (cv_09_detection). Use `wandb login --relogin` to force relogin


2023-01-04 07:39:49,878 - mmseg - INFO - Loaded 655 images



[Uniform Soup]
load checkpoint from local path: /opt/ml/input/level2_semanticsegmentation_cv-level2-cv-09/mmsegmentation/work_dirs/segformer_mit-b0_512x512_160k_ade20k/iter_96000.pth
load checkpoint from local path: /opt/ml/input/level2_semanticsegmentation_cv-level2-cv-09/mmsegmentation/work_dirs/segformer_mit-b0_512x512_160k_ade20k/iter_108000.pth
load checkpoint from local path: /opt/ml/input/level2_semanticsegmentation_cv-level2-cv-09/mmsegmentation/work_dirs/segformer_mit-b0_512x512_160k_ade20k/iter_120000.pth
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 655/655, 24.2 task/s, elapsed: 27s, ETA:     0sper class results:

+---------------+-------+-------+
|     Class     |  IoU  |  Acc  |
+---------------+-------+-------+
|   Background  | 94.76 | 97.53 |
| General trash | 42.71 | 53.53 |
|     Paper     | 72.07 | 85.16 |
|   Paper pack  | 41.57 | 54.18 |
|     Metal     | 40.58 |  47.9 |
|     Glass     | 51.53 |  64.5 |
|    Plastic    | 39.66 | 61.01 |
|   Styrofoam   | 71.02 | 76.62 |
|  Pl

2. Greedy Soup (uniform weight update)

In [100]:
################ save dir path 적기 ################
save_dir_path = './soup/'
name = 'test' # soup 이름 적기
################ save dir path 적기 ################

print("[Greedy Soup (uniform weight update)]")
greedy_model, checkpoint = greedy_soup(cfg, model, checkpoint_paths, device = device)
greedy_dict = checkpoint
greedy_dict['state_dict'] = greedy_model.state_dict()
torch.save(greedy_dict, save_dir_path+f'greedy_model_soup_{name}.pth')


[Greedy Soup (uniform weight update)]
loading annotations into memory...
Done (t=0.99s)
creating index...
index created!
load checkpoint from local path: /opt/ml/work_dirs/segformer_mit-b5_8x1_1024x1024_160k_cityscapes/iter_16000.pth
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 655/655, 4.8 task/s, elapsed: 137s, ETA:     0sper class results:

+---------------+-------+-------+
|     Class     |  IoU  |  Acc  |
+---------------+-------+-------+
|   Backgroud   | 94.03 | 97.35 |
| General trash | 18.16 | 18.55 |
|     Paper     | 61.73 | 91.95 |
|   Paper pack  | 15.33 | 21.68 |
|     Metal     | 27.56 | 65.48 |
|     Glass     | 26.49 | 31.41 |
|    Plastic    |  32.1 | 38.99 |
|   Styrofoam   | 48.67 | 50.09 |
|  Plastic bag  |  79.5 | 84.76 |
|    Battery    |  0.0  |  0.0  |
|    Clothing   | 38.98 |  50.9 |
+---------------+-------+-------+
Summary:

+-------+-------+------+
|  aAcc |  mIoU | mAcc |
+-------+-------+------+
| 89.72 | 40.23 | 50.1 |
+-------+-------+------+
리스트에 0번째 mIoU 0.4023저장